In [128]:
import os
import numpy as np
import seaborn as sns
import pandas as pd
import subprocess
from dtw import correlate
import json
import librosa
import librosa.display
import IPython.display as ipd
import math
import csv
import matplotlib.pyplot as plt


# Breathing
#instrument_key = 'breathing'
#AUDIO_DURATION_MS = [0, 8 * 1000]
#is_breathing = True

# Ambient
instrument_key = 'ambient'
AUDIO_DURATION_MS = [4 * 1000, 8 * 1000]
is_breathing = False


MAX_SAMPLE_COMPARISON_COUNT = 10000

apppath = os.path.expanduser(os.getcwd() + '/')
#mediapath = os.path.expanduser('/mnt/freesound-crawler/')
#mediapath = os.path.expanduser('/Volumes/LaCie/freesound-crawler/')
mediapath = os.path.expanduser('/Users/andrusasumets/Projects/freesound-crawler/')
processedpath = os.path.join(mediapath, 'processed/')
soxipath = os.path.join(processedpath, 'soxi/')
wavpath = os.path.join(processedpath, 'wav/')
audiopath = os.path.join(processedpath, 'audio/')
convertedpath = os.path.join(processedpath, 'converted/')
normalizedpath = os.path.join(processedpath, 'normalized/')
silencedpath = os.path.join(processedpath, 'silenced/')
dtwpath = os.path.join(processedpath, 'dtw/')
preselectedpath = os.path.join(processedpath, 'preselected/' + instrument_key + '/')
excludedpath = os.path.join(processedpath, 'excluded/' + instrument_key + '/')
extensionpath = os.path.join(apppath, 'Extension/data/distances/')
extensionsamplepath = os.path.join('Extension/data/', 'samples/')
librarypath = os.path.join(mediapath, 'downloads/')
metadatapath = os.path.join(mediapath, 'metadata/')
metadatacsvpath = os.path.join(processedpath, 'metadata.csv')
allowedfileformats = ['wav', 'aiff', 'flac', 'ogg', 'mp3', 'm4a']
processfileformat = 'wav'
exportfileformat = 'm4a'

for p in [
    processedpath,
    soxipath,
    wavpath,
    audiopath,
    convertedpath,
    normalizedpath,
    silencedpath,
    dtwpath,
    preselectedpath,
    excludedpath,
    extensionpath,
    extensionsamplepath
]:
    if not os.path.exists(p):
        os.makedirs(p)

In [129]:
def read_csv(filename):
    with open(filename) as f:
        file_data = csv.reader(f)
        headers = next(file_data)
        return [dict(zip(headers, i)) for i in file_data] 
    
def process(command, verbose=False):
    if verbose:
        print(' '.join(command))
    subprocess.Popen(command, shell=False)
    
def get_fileformat(filename):
    return filename.split('.')[len(filename.split('.')) - 1]

def create_or_read_file(command, outfilename):
    if os.path.exists(outfilename):
        f = open(outfilename, "r")
        out = ''.join(f.readlines())
        f.close()
        return out
    else:
        try:
            out = str(subprocess.check_output(command).decode(encoding="utf-8")).strip().replace('\\n', '').replace("'", "")

            with open(outfilename, 'w') as file:
                file.write(out)

            return out
        except Exception as e:
            return 

In [130]:
ids = [f for f in os.listdir(metadatapath)]
print(len(ids))

if not os.path.exists(metadatacsvpath):
    fieldnames = ['id', 'duration', 'downloads', 'rating', 'numratings', 'tags']
    rows = []

    for id in ids:        
        with open(metadatapath + "/" + id) as file:
            try:
                id = int(id)
                line = file.readlines()[0]
                arguments = line.split('|')
                duration = int(arguments[0].split('duration:')[1])
                downloads = int(arguments[1].split('downloads:')[1])
                rating = int(arguments[2].split('rating:')[1])
                numratings = int(arguments[3].split('numratings:')[1])
                tags = arguments[4].split('tags:')[1]
                row = {
                    'id': id,
                    'duration': duration,
                    'downloads': downloads,
                    'rating': rating,
                    'numratings': numratings,
                    'tags': tags
                }
                rows.append(row)
            except Exception as e:
                print(e)

    with open(metadatacsvpath, 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames = fieldnames)
        writer.writeheader()
        writer.writerows(rows)

metadata = read_csv(metadatacsvpath)

In [131]:
def parse_ids(path, fileformat):
    result = [f for f in os.listdir(path)]
    result = [f for f in result if f.endswith(fileformat)]
    result = [f.split('-')[0] for f in result]
    result = [f.split('.')[0] for f in result]
    return result

preselected = parse_ids(preselectedpath, exportfileformat)
excluded = parse_ids(excludedpath, exportfileformat)

print('preselected', len(preselected))
print('excluded', len(excluded))

In [132]:
DOWNLOADS_THRESHOLD = 10
RATING_THRESHOLD = 90
MIN_NUMRATINGS = 3

scored = []
for sample in metadata:            
    id = sample['id']
    
    if id in excluded and id not in preselected:
        continue

    duration = int(sample['duration'])
    downloads = int(sample['downloads'])
    rating = int(sample['rating'])
    numratings = int(sample['numratings'])
    tags = sample['tags']

    if len(tags) > 0 and downloads >= DOWNLOADS_THRESHOLD and rating >= RATING_THRESHOLD and numratings >= MIN_NUMRATINGS:
        row = [id, duration, downloads, rating, numratings, tags, rating * downloads]
        scored.append(row)

print(len(scored))

scored.sort(key=lambda x: int(x[6]))
scored.reverse()

print(scored[0])

In [133]:
samples = {}

for row in scored:
    id = row[0]
    samples[id] = {
        'duration': row[1],
        'downloads': row[2],
        'rating': row[3],
        'numratings': row[4],
        'tags': row[5],
        'score': row[6],
    }
    
print(len(list(samples)))
print(samples[scored[0][0]])

In [134]:
all_filenames = [f for f in os.listdir(librarypath)]
fileformats = {}
for filename in all_filenames:
    fileformat = get_fileformat(filename)
    
    if fileformat not in fileformats:
        fileformats[fileformat] = 0
         
    fileformats[fileformat] = fileformats[fileformat] + 1
    
print('fileformats', fileformats)
print('all_filenames:', len(all_filenames))

filenames = [f for f in os.listdir(librarypath) if f.endswith(tuple(allowedfileformats))]
filenames = [f for f in filenames if f.count('__') == 2]
filtered = []

for filename in filenames:
    id = filename.split('__')[0]
    
    if id in excluded:
        continue
    
    for row in scored:
        if row[0] == id:
            filtered.append(filename)
            
filenames = filtered
print('filtered filenames', len(filenames))

In [135]:
exportedfilenames = [f for f in os.listdir(audiopath) if f.endswith(exportfileformat)]

test = [f.split('__')[0] for f in filenames]

exported = []
for filename in filenames:
    try:
        fileartist = filename.split("__")[1].split("__")[0]
        fileid = filename.split('__')[0]
        audiofilename = fileid + '.' + processfileformat
        exportfilename = fileid + '.' + exportfileformat
        
        if samples[fileid]['duration'] < AUDIO_DURATION_MS[0]:
            continue
        
        if samples[fileid]['duration'] > AUDIO_DURATION_MS[1]:
            continue
        
        if exportfilename not in exportedfilenames:
            process(['ffmpeg', '-loglevel', 'error', '-i', librarypath + filename, '-acodec', 'pcm_s16le', '-ar', '44100', '-threads', '6', convertedpath + audiofilename])
            process(['ffmpeg-normalize', convertedpath + audiofilename, '-o', normalizedpath + audiofilename, '-f'])
            process(['ffmpeg', '-loglevel', 'error', '-i', normalizedpath + audiofilename, '-af', 'silenceremove=1:0:-50dB', '-threads', '6', silencedpath + audiofilename])
            process(['ffmpeg', '-loglevel', 'error', '-i', silencedpath + audiofilename, '-c:a', 'aac', '-ar', '44100', '-threads', '6', audiopath + exportfilename])
            process(['ffmpeg', '-loglevel', 'error', '-i', silencedpath + audiofilename, '-acodec', 'pcm_s16le', '-ac', '1', '-ar', '100', '-threads', '6', wavpath + fileid + '.wav'])

        exported.append(fileid)
    except Exception as e:
        print(e)

for id in preselected:
    if id not in exported:
        exported.append(id)

print(len(exported))

In [136]:
FREQUENCY_MULTIPLIER = 0.5

saved = []
for leftid in exported:
    if leftid in saved:
        continue

    source = audiopath + leftid + '.' + exportfileformat
    target = extensionsamplepath + leftid + '.' + exportfileformat
    target_breathe_in = extensionsamplepath + leftid + '-breathe-in.' + exportfileformat
    target_breathe_out = extensionsamplepath + leftid + '-breathe-out.' + exportfileformat

    try:
        if is_breathing:
            if not os.path.isfile(target_breathe_out):
                process(['cp', source, target_breathe_out])

            if not os.path.isfile(target_breathe_in):
                process(['ffmpeg', '-i', source, '-af', 'asetrate=44100*' + str(FREQUENCY_MULTIPLIER) + ',aresample=44100,atempo=1/' + str(FREQUENCY_MULTIPLIER), target_breathe_in])
        else:
            if not os.path.isfile(target):
                process(['cp', source, target])
                
                
        saved.append(leftid)
 
    except Exception as e:
        print(e)

In [137]:
#EXPORTED_COUNT = 10
EXPORTED_COUNT = len(exported)

df = pd.DataFrame()
out_distances = {}
full_distances = []

for leftfilename in exported[0:EXPORTED_COUNT]:
    leftid = leftfilename.split('.')[0]
    out_distances[leftid] = []
    
    for righti, rightfilename in enumerate(exported[0:EXPORTED_COUNT]):
        if (len(full_distances) % 100) == 0 and righti == 0:
            print(str(len(full_distances)) + " / " + str(len(exported) * MAX_SAMPLE_COMPARISON_COUNT))
                
        if righti >= MAX_SAMPLE_COMPARISON_COUNT:
            continue
        
        if leftfilename == rightfilename:
            continue
            
        rightid = rightfilename.split('.')[0]
            
        try:
            correlation = correlate(wavpath, leftid, rightid, 'wav', dtwpath)
            
            if math.isinf(float(correlation)):
                continue
                
            if math.isnan(float(correlation)):
                continue

            df.loc[leftid, rightid] = correlation
            df.loc[rightid, leftid] = correlation
            
            out_distances[leftid].append([rightid, correlation])
            full_distances.append([correlation, leftid, rightid])

        except Exception as e:
            pass


for filename in out_distances:
    out_distances[filename] = sorted(out_distances[filename])
    
full_distances = sorted(full_distances)
print(len(full_distances))

with open(extensionpath + instrument_key + '.json', 'w') as fp:
    json.dump(out_distances, fp)

In [138]:
plot = sns.heatmap(df, xticklabels=False, yticklabels=False)
handles, labels = plot.get_legend_handles_labels()
plot.legend(reversed(handles), reversed(labels), title='', loc="lower right")
fig = plot.get_figure()
fig.tight_layout()

In [139]:
def show_sample(comparison):
    distance, leftid, rightid = comparison
    print(distance, leftid, rightid)

    data, rate = librosa.core.load(audiopath + leftid + '.' + exportfileformat)
    ipd.display(ipd.Audio(data, rate=rate))

    data, rate = librosa.core.load(audiopath + rightid + '.' + exportfileformat)
    ipd.display(ipd.Audio(data, rate=rate))

display_count = 10
for i in range(display_count):
    if (i % 2) == 0:
        show_sample(full_distances[i])
for i in range(display_count):
    if (i % 2) == 0:
        show_sample(full_distances[i - display_count])

In [140]:
if is_breathing:
    plots = saved[0:EXPORTED_COUNT]
    plt.rcParams.update({'font.size': 6})
    fig, ax = plt.subplots(nrows=len(plots), figsize=(10, EXPORTED_COUNT * 2))

    def waveform(path, ax):
        data, rate = librosa.core.load(path)
        ax.set_title('⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀' + path)
        ax.label_outer()
        librosa.display.waveshow(data, sr=rate, ax=ax, alpha=0.75)

    for idx, id in enumerate(plots):
        breathe_in = extensionsamplepath + id + '-breathe-in.' + exportfileformat

        if is_breathing:
            breathe_out = extensionsamplepath + id + '-breathe-out.' + exportfileformat

        waveform(breathe_in, ax[idx])
        waveform(breathe_out, ax[idx])